<a href="https://colab.research.google.com/github/RakshithJKashyap/PySpark/blob/main/consecutiveMarksPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from pyspark.sql import SparkSession
import matplotlib.pyplot as plt
import pandas as pd
import sys
import os
from pyspark.ml.feature import StringIndexer
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import OneHotEncoder

In [2]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 35 kB/s 
     |████████████████████████████████| 198 kB 58.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=5b740d182dd9bf2e999224e440418890e6e72d8babe3387e06d053233cca5b16
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [4]:
spark = SparkSession.builder.appName('Practise').getOrCreate()

In [5]:
df = spark.read.csv('/content/Results_Data.csv',header=True,inferSchema=True)
df.show()

+----------+--------+---+---+---+------+-----+-----+--------+
|       USN|   SCODE|CIE|SEE|TOT|RESULT|GRADE| DATE|ExamType|
+----------+--------+---+---+---+------+-----+-----+--------+
|4JN18CS001| 18MAT11| 32| 41| 73|     P|    8|12019| Regular|
|4JN18CS001| 18PHY12| 29| 47| 76|     P|    8|12019| Regular|
|4JN18CS001| 18ELE13| 25| 43| 68|     P|    7|12019| Regular|
|4JN18CS001| 18CIV14| 31| 26| 57|     P|    6|12019| Regular|
|4JN18CS001|18EGDL15| 31| 56| 87|     P|    9|12019| Regular|
|4JN18CS001|18PHYL16| 32| 40| 72|     P|    8|12019| Regular|
|4JN18CS001|18ELEL17| 34| 52| 86|     P|    9|12019| Regular|
|4JN18CS001| 18EGH18| 27| 32| 59|     P|    6|12019| Regular|
|4JN18CS002| 18MAT11| 33| 51| 84|     P|    9|12019| Regular|
|4JN18CS002| 18PHY12| 36| 45| 81|     P|    9|12019| Regular|
|4JN18CS002| 18ELE13| 37| 31| 68|     P|    7|12019| Regular|
|4JN18CS002| 18CIV14| 34| 29| 63|     P|    7|12019| Regular|
|4JN18CS002|18EGDL15| 34| 55| 89|     P|    9|12019| Regular|
|4JN18CS

In [6]:
import string
encoded1= {}
count =0
for i in string.ascii_uppercase:
    if i not in encoded1:
        encoded1[i]= count
    count = count +1
print (encoded1)

{'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9, 'K': 10, 'L': 11, 'M': 12, 'N': 13, 'O': 14, 'P': 15, 'Q': 16, 'R': 17, 'S': 18, 'T': 19, 'U': 20, 'V': 21, 'W': 22, 'X': 23, 'Y': 24, 'Z': 25}


In [7]:
data_itr = df.rdd.toLocalIterator()
encoded = []

def replace_all1(text, encoded1):
    for i, j in encoded1.items():
        text = text.replace(i, str(j))
    return text

print(encoded1)

for i in data_itr:
    a =i['USN'] 
    b = i['SCODE']
    c = i['DATE']
    encoded.append({'USN1':a,'encodedUSN':int(replace_all1(a,encoded1)),'SCODE1':b,'DATE1':c})
        
print(encoded)

{'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9, 'K': 10, 'L': 11, 'M': 12, 'N': 13, 'O': 14, 'P': 15, 'Q': 16, 'R': 17, 'S': 18, 'T': 19, 'U': 20, 'V': 21, 'W': 22, 'X': 23, 'Y': 24, 'Z': 25}
[{'USN1': '4JN18CS001', 'encodedUSN': 491318218001, 'SCODE1': '18MAT11', 'DATE1': '12019'}, {'USN1': '4JN18CS001', 'encodedUSN': 491318218001, 'SCODE1': '18PHY12', 'DATE1': '12019'}, {'USN1': '4JN18CS001', 'encodedUSN': 491318218001, 'SCODE1': '18ELE13', 'DATE1': '12019'}, {'USN1': '4JN18CS001', 'encodedUSN': 491318218001, 'SCODE1': '18CIV14', 'DATE1': '12019'}, {'USN1': '4JN18CS001', 'encodedUSN': 491318218001, 'SCODE1': '18EGDL15', 'DATE1': '12019'}, {'USN1': '4JN18CS001', 'encodedUSN': 491318218001, 'SCODE1': '18PHYL16', 'DATE1': '12019'}, {'USN1': '4JN18CS001', 'encodedUSN': 491318218001, 'SCODE1': '18ELEL17', 'DATE1': '12019'}, {'USN1': '4JN18CS001', 'encodedUSN': 491318218001, 'SCODE1': '18EGH18', 'DATE1': '12019'}, {'USN1': '4JN18CS002', 'encodedUSN': 491318

In [8]:
dataframe = spark.createDataFrame(encoded)
dataframe.show()
dataframe.count()

+-----+--------+----------+------------+
|DATE1|  SCODE1|      USN1|  encodedUSN|
+-----+--------+----------+------------+
|12019| 18MAT11|4JN18CS001|491318218001|
|12019| 18PHY12|4JN18CS001|491318218001|
|12019| 18ELE13|4JN18CS001|491318218001|
|12019| 18CIV14|4JN18CS001|491318218001|
|12019|18EGDL15|4JN18CS001|491318218001|
|12019|18PHYL16|4JN18CS001|491318218001|
|12019|18ELEL17|4JN18CS001|491318218001|
|12019| 18EGH18|4JN18CS001|491318218001|
|12019| 18MAT11|4JN18CS002|491318218002|
|12019| 18PHY12|4JN18CS002|491318218002|
|12019| 18ELE13|4JN18CS002|491318218002|
|12019| 18CIV14|4JN18CS002|491318218002|
|12019|18EGDL15|4JN18CS002|491318218002|
|12019|18PHYL16|4JN18CS002|491318218002|
|12019|18ELEL17|4JN18CS002|491318218002|
|12019| 18EGH18|4JN18CS002|491318218002|
|12019| 18MAT11|4JN18CS003|491318218003|
|12019| 18PHY12|4JN18CS003|491318218003|
|12019| 18ELE13|4JN18CS003|491318218003|
|12019| 18CIV14|4JN18CS003|491318218003|
+-----+--------+----------+------------+
only showing top

15000

In [9]:
new = df.join(dataframe,(df.USN ==  dataframe.USN1) & (df.SCODE ==dataframe.SCODE1) & (df.DATE == dataframe.DATE1))
new.show()
new.drop('USN1').show()
new.count()

+----------+--------+---+---+---+------+-----+-----+--------+-----+--------+----------+------------+
|       USN|   SCODE|CIE|SEE|TOT|RESULT|GRADE| DATE|ExamType|DATE1|  SCODE1|      USN1|  encodedUSN|
+----------+--------+---+---+---+------+-----+-----+--------+-----+--------+----------+------------+
|4JN18CS001| 18MAT11| 32| 41| 73|     P|    8|12019| Regular|12019| 18MAT11|4JN18CS001|491318218001|
|4JN18CS001| 18PHY12| 29| 47| 76|     P|    8|12019| Regular|12019| 18PHY12|4JN18CS001|491318218001|
|4JN18CS001| 18ELE13| 25| 43| 68|     P|    7|12019| Regular|12019| 18ELE13|4JN18CS001|491318218001|
|4JN18CS001| 18CIV14| 31| 26| 57|     P|    6|12019| Regular|12019| 18CIV14|4JN18CS001|491318218001|
|4JN18CS001|18EGDL15| 31| 56| 87|     P|    9|12019| Regular|12019|18EGDL15|4JN18CS001|491318218001|
|4JN18CS001|18PHYL16| 32| 40| 72|     P|    8|12019| Regular|12019|18PHYL16|4JN18CS001|491318218001|
|4JN18CS001|18ELEL17| 34| 52| 86|     P|    9|12019| Regular|12019|18ELEL17|4JN18CS001|4913

15000

In [10]:
new.createOrReplaceTempView("df2_view")

dffinal = spark.sql(
    """
    SELECT encodedUSN,TOT,
    (CASE
    WHEN SCODE LIKE '%1' THEN 1
    WHEN SCODE LIKE '%2' THEN 2
    WHEN SCODE LIKE '%3' THEN 3
    WHEN SCODE LIKE '%4' THEN 4
    WHEN SCODE LIKE '%5' THEN 5
    WHEN SCODE LIKE '%6' THEN 6
    WHEN SCODE LIKE '%7' THEN 7
    WHEN SCODE LIKE '%8' THEN 8
    WHEN SCODE LIKE '%9' THEN 9
    END) AS SCODEFINAL,
    (CASE
    WHEN DATE LIKE '%19' THEN 19
    WHEN DATE LIKE '%20' THEN 20
    END) AS NEWDATE
    
    FROM df2_view
    """
)

dffinal.show()

+------------+---+----------+-------+
|  encodedUSN|TOT|SCODEFINAL|NEWDATE|
+------------+---+----------+-------+
|491318218001| 73|         1|     19|
|491318218001| 76|         2|     19|
|491318218001| 68|         3|     19|
|491318218001| 57|         4|     19|
|491318218001| 87|         5|     19|
|491318218001| 72|         6|     19|
|491318218001| 86|         7|     19|
|491318218001| 59|         8|     19|
|491318218002| 84|         1|     19|
|491318218002| 81|         2|     19|
|491318218002| 68|         3|     19|
|491318218002| 63|         4|     19|
|491318218002| 89|         5|     19|
|491318218002| 75|         6|     19|
|491318218002| 84|         7|     19|
|491318218002| 72|         8|     19|
|491318218003| 81|         1|     19|
|491318218003| 73|         2|     19|
|491318218003| 81|         3|     19|
|491318218003| 72|         4|     19|
+------------+---+----------+-------+
only showing top 20 rows



In [11]:
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=['encodedUSN','NEWDATE','SCODEFINAL'],outputCol="NEW")
output=featureassembler.transform(dffinal)
output.show()

+------------+---+----------+-------+--------------------+
|  encodedUSN|TOT|SCODEFINAL|NEWDATE|                 NEW|
+------------+---+----------+-------+--------------------+
|491318218001| 73|         1|     19|[4.91318218001E11...|
|491318218001| 76|         2|     19|[4.91318218001E11...|
|491318218001| 68|         3|     19|[4.91318218001E11...|
|491318218001| 57|         4|     19|[4.91318218001E11...|
|491318218001| 87|         5|     19|[4.91318218001E11...|
|491318218001| 72|         6|     19|[4.91318218001E11...|
|491318218001| 86|         7|     19|[4.91318218001E11...|
|491318218001| 59|         8|     19|[4.91318218001E11...|
|491318218002| 84|         1|     19|[4.91318218002E11...|
|491318218002| 81|         2|     19|[4.91318218002E11...|
|491318218002| 68|         3|     19|[4.91318218002E11...|
|491318218002| 63|         4|     19|[4.91318218002E11...|
|491318218002| 89|         5|     19|[4.91318218002E11...|
|491318218002| 75|         6|     19|[4.91318218002E11..

In [58]:
df2 = output.select('encodedUSN','NEW','TOT')
limited = df2.limit(4924)

In [59]:
train, test = limited.randomSplit([0.9, 0.1])

In [60]:
regressor=LinearRegression(featuresCol='NEW', labelCol='TOT')
regressor=regressor.fit(train)

In [61]:
predictions = regressor.transform(test)
predictions.select(['NEW','TOT','prediction']).show(25)

+--------------------+---+------------------+
|                 NEW|TOT|        prediction|
+--------------------+---+------------------+
|[4.91314218002E11...| 31| 62.98013417639595|
|[4.91314218002E11...| 92| 70.18929687108903|
|[4.91314218092E11...| 91| 71.99161128092965|
|[4.91315218001E11...| 63|59.639287585087004|
|[4.91315218001E11...| 94| 68.65074095345335|
|[4.91315218002E11...| 44|61.441578522499185|
|[4.91315218003E11...| 59|63.243869459896814|
|[4.91315218003E11...| 74| 65.04616013357008|
|[4.91315218004E11...| 53|   65.046160397309|
|[4.91315218005E11...| 72| 61.44157931370137|
|[4.91315218005E11...| 98| 72.25532335575554|
|[4.91315218007E11...| 64| 61.44157984116464|
|[4.91315218007E11...| 97| 68.65074253585772|
|[4.91315218008E11...| 47| 61.44158010490355|
|[4.91315218008E11...| 95|  70.4530334732699|
|[4.91315218009E11...| 70| 61.44158036864246|
|[4.91315218009E11...| 88|   65.046161715989|
|[4.91315218009E11...| 92| 68.65074306333554|
|[4.91315218011E11...| 61| 65.0461

In [74]:
dftest = spark.read.csv('/content/new1.csv',header=True,inferSchema=True)
dftest.show()
dftest = dftest.limit(15000)

+----------+--------+---+----+
|       USN|   SCODE|TOT|DATE|
+----------+--------+---+----+
|4JN18CS001| 18MAT11| 73|2021|
|4JN18CS001| 18PHY12| 76|2021|
|4JN18CS001| 18ELE13| 68|2021|
|4JN18CS001| 18CIV14| 57|2021|
|4JN18CS001|18EGDL15| 87|2021|
|4JN18CS001|18PHYL16| 72|2021|
|4JN18CS001|18ELEL17| 86|2021|
|4JN18CS001| 18EGH18| 59|2021|
|4JN18CS002| 18MAT11| 84|2021|
|4JN18CS002| 18PHY12| 81|2021|
|4JN18CS002| 18ELE13| 68|2021|
|4JN18CS002| 18CIV14| 63|2021|
|4JN18CS002|18EGDL15| 89|2021|
|4JN18CS002|18PHYL16| 75|2021|
|4JN18CS002|18ELEL17| 84|2021|
|4JN18CS002| 18EGH18| 72|2021|
|4JN18CS003| 18MAT11| 81|2021|
|4JN18CS003| 18PHY12| 73|2021|
|4JN18CS003| 18ELE13| 81|2021|
|4JN18CS003| 18CIV14| 72|2021|
+----------+--------+---+----+
only showing top 20 rows



In [75]:
data_itr1 = dftest.rdd.toLocalIterator()
encoded3 = []

def replace_all1(text, encoded1):
    for i, j in encoded1.items():
        text = text.replace(i, str(j))
    return text

print(encoded3)

for i in data_itr1:
    a =i['USN'] 
    b = i['SCODE']
    c = i['TOT']
    encoded3.append({'USN1':a,'encodedUSN':int(replace_all1(a,encoded1)),'SCODE1':b,'TOT1':c})
        
print(encoded3)

[]
[{'USN1': '4JN18CS001', 'encodedUSN': 491318218001, 'SCODE1': '18MAT11', 'TOT1': 73}, {'USN1': '4JN18CS001', 'encodedUSN': 491318218001, 'SCODE1': '18PHY12', 'TOT1': 76}, {'USN1': '4JN18CS001', 'encodedUSN': 491318218001, 'SCODE1': '18ELE13', 'TOT1': 68}, {'USN1': '4JN18CS001', 'encodedUSN': 491318218001, 'SCODE1': '18CIV14', 'TOT1': 57}, {'USN1': '4JN18CS001', 'encodedUSN': 491318218001, 'SCODE1': '18EGDL15', 'TOT1': 87}, {'USN1': '4JN18CS001', 'encodedUSN': 491318218001, 'SCODE1': '18PHYL16', 'TOT1': 72}, {'USN1': '4JN18CS001', 'encodedUSN': 491318218001, 'SCODE1': '18ELEL17', 'TOT1': 86}, {'USN1': '4JN18CS001', 'encodedUSN': 491318218001, 'SCODE1': '18EGH18', 'TOT1': 59}, {'USN1': '4JN18CS002', 'encodedUSN': 491318218002, 'SCODE1': '18MAT11', 'TOT1': 84}, {'USN1': '4JN18CS002', 'encodedUSN': 491318218002, 'SCODE1': '18PHY12', 'TOT1': 81}, {'USN1': '4JN18CS002', 'encodedUSN': 491318218002, 'SCODE1': '18ELE13', 'TOT1': 68}, {'USN1': '4JN18CS002', 'encodedUSN': 491318218002, 'SCODE1

In [76]:
dataframe1 = spark.createDataFrame(encoded3)
dataframe1.show()
dataframe1.count()

+--------+----+----------+------------+
|  SCODE1|TOT1|      USN1|  encodedUSN|
+--------+----+----------+------------+
| 18MAT11|  73|4JN18CS001|491318218001|
| 18PHY12|  76|4JN18CS001|491318218001|
| 18ELE13|  68|4JN18CS001|491318218001|
| 18CIV14|  57|4JN18CS001|491318218001|
|18EGDL15|  87|4JN18CS001|491318218001|
|18PHYL16|  72|4JN18CS001|491318218001|
|18ELEL17|  86|4JN18CS001|491318218001|
| 18EGH18|  59|4JN18CS001|491318218001|
| 18MAT11|  84|4JN18CS002|491318218002|
| 18PHY12|  81|4JN18CS002|491318218002|
| 18ELE13|  68|4JN18CS002|491318218002|
| 18CIV14|  63|4JN18CS002|491318218002|
|18EGDL15|  89|4JN18CS002|491318218002|
|18PHYL16|  75|4JN18CS002|491318218002|
|18ELEL17|  84|4JN18CS002|491318218002|
| 18EGH18|  72|4JN18CS002|491318218002|
| 18MAT11|  81|4JN18CS003|491318218003|
| 18PHY12|  73|4JN18CS003|491318218003|
| 18ELE13|  81|4JN18CS003|491318218003|
| 18CIV14|  72|4JN18CS003|491318218003|
+--------+----+----------+------------+
only showing top 20 rows



15000

In [77]:
new5 = dftest.join(dataframe1,(dftest.USN ==  dataframe1.USN1) & (dftest.SCODE ==dataframe1.SCODE1) )
new5.show()
new5.drop('USN1').show()
new5.count()

+----------+--------+---+----+--------+----+----------+------------+
|       USN|   SCODE|TOT|DATE|  SCODE1|TOT1|      USN1|  encodedUSN|
+----------+--------+---+----+--------+----+----------+------------+
|4JN18CS001| 18MAT11| 73|2021| 18MAT11|  73|4JN18CS001|491318218001|
|4JN18CS001| 18PHY12| 76|2021| 18PHY12|  76|4JN18CS001|491318218001|
|4JN18CS001| 18ELE13| 68|2021| 18ELE13|  68|4JN18CS001|491318218001|
|4JN18CS001| 18CIV14| 57|2021| 18CIV14|  57|4JN18CS001|491318218001|
|4JN18CS001|18EGDL15| 87|2021|18EGDL15|  87|4JN18CS001|491318218001|
|4JN18CS001|18PHYL16| 72|2021|18PHYL16|  72|4JN18CS001|491318218001|
|4JN18CS001|18ELEL17| 86|2021|18ELEL17|  86|4JN18CS001|491318218001|
|4JN18CS001| 18EGH18| 59|2021| 18EGH18|  59|4JN18CS001|491318218001|
|4JN18CS002| 18MAT11| 84|2021| 18MAT11|  84|4JN18CS002|491318218002|
|4JN18CS002| 18PHY12| 81|2021| 18PHY12|  81|4JN18CS002|491318218002|
|4JN18CS002| 18ELE13| 68|2021| 18ELE13|  68|4JN18CS002|491318218002|
|4JN18CS002| 18CIV14| 63|2021| 18C

15670

In [79]:
new5.createOrReplaceTempView("df7_view")

dffinal7 = spark.sql(
    """
    SELECT encodedUSN,TOT,
    (CASE
    WHEN SCODE LIKE '%1' THEN 1
    WHEN SCODE LIKE '%2' THEN 2
    WHEN SCODE LIKE '%3' THEN 3
    WHEN SCODE LIKE '%4' THEN 4
    WHEN SCODE LIKE '%5' THEN 5
    WHEN SCODE LIKE '%6' THEN 6
    WHEN SCODE LIKE '%7' THEN 7
    WHEN SCODE LIKE '%8' THEN 8
    WHEN SCODE LIKE '%9' THEN 9
    END) AS SCODEFINAL,
    (CASE
    WHEN DATE LIKE '%19' THEN 19
    WHEN DATE LIKE '%21' THEN 21
    END) AS NEWDATE
    
    FROM df7_view
    """
)

dffinal7.show()

+------------+---+----------+-------+
|  encodedUSN|TOT|SCODEFINAL|NEWDATE|
+------------+---+----------+-------+
|491318218001| 73|         1|     21|
|491318218001| 76|         2|     21|
|491318218001| 68|         3|     21|
|491318218001| 57|         4|     21|
|491318218001| 87|         5|     21|
|491318218001| 72|         6|     21|
|491318218001| 86|         7|     21|
|491318218001| 59|         8|     21|
|491318218002| 84|         1|     21|
|491318218002| 81|         2|     21|
|491318218002| 68|         3|     21|
|491318218002| 63|         4|     21|
|491318218002| 89|         5|     21|
|491318218002| 75|         6|     21|
|491318218002| 84|         7|     21|
|491318218002| 72|         8|     21|
|491318218003| 81|         1|     21|
|491318218003| 73|         2|     21|
|491318218003| 81|         3|     21|
|491318218003| 72|         4|     21|
+------------+---+----------+-------+
only showing top 20 rows



In [80]:
featureassembler=VectorAssembler(inputCols=['encodedUSN','NEWDATE','SCODEFINAL'],outputCol="NEW")
output1=featureassembler.transform(dffinal7)
output1.show()

+------------+---+----------+-------+--------------------+
|  encodedUSN|TOT|SCODEFINAL|NEWDATE|                 NEW|
+------------+---+----------+-------+--------------------+
|491318218001| 73|         1|     21|[4.91318218001E11...|
|491318218001| 76|         2|     21|[4.91318218001E11...|
|491318218001| 68|         3|     21|[4.91318218001E11...|
|491318218001| 57|         4|     21|[4.91318218001E11...|
|491318218001| 87|         5|     21|[4.91318218001E11...|
|491318218001| 72|         6|     21|[4.91318218001E11...|
|491318218001| 86|         7|     21|[4.91318218001E11...|
|491318218001| 59|         8|     21|[4.91318218001E11...|
|491318218002| 84|         1|     21|[4.91318218002E11...|
|491318218002| 81|         2|     21|[4.91318218002E11...|
|491318218002| 68|         3|     21|[4.91318218002E11...|
|491318218002| 63|         4|     21|[4.91318218002E11...|
|491318218002| 89|         5|     21|[4.91318218002E11...|
|491318218002| 75|         6|     21|[4.91318218002E11..

In [81]:
testfinal = output1.select('NEW','TOT')

In [83]:
predictions = regressor.transform(testfinal)
predictions.select(['NEW','TOT','prediction']).show(25)

+--------------------+---+------------------+
|                 NEW|TOT|        prediction|
+--------------------+---+------------------+
|[4.91318218001E11...| 73|60.430492644387414|
|[4.91318218001E11...| 76| 62.23278331806068|
|[4.91318218001E11...| 68| 64.03507399173395|
|[4.91318218001E11...| 57| 65.83736466540722|
|[4.91318218001E11...| 87| 67.63965533908049|
|[4.91318218001E11...| 72| 69.44194601275376|
|[4.91318218001E11...| 86| 71.24423668642703|
|[4.91318218001E11...| 59| 73.04652736011485|
|[4.91318218002E11...| 84| 60.43049290811177|
|[4.91318218002E11...| 81| 62.23278358178504|
|[4.91318218002E11...| 68| 64.03507425545831|
|[4.91318218002E11...| 63| 65.83736492913158|
|[4.91318218002E11...| 89| 67.63965560280485|
|[4.91318218002E11...| 75| 69.44194627647812|
|[4.91318218002E11...| 84| 71.24423695015139|
|[4.91318218002E11...| 72| 73.04652762383921|
|[4.91318218003E11...| 81|60.430493171850685|
|[4.91318218003E11...| 73|62.232783845523954|
|[4.91318218003E11...| 81| 64.0350

In [84]:
predictions.count()

15670

In [85]:
mPath = "/content/drive/MyDrive/model2"
regressor.write().overwrite().save(mPath)